## Soil Sensor deployment at farm scale: a feasibility analysis
_Q: Can such consumer technology be used for remote sensing at farm scale? With positive ROI?_

![img source](https://i.imgur.com/bknlfCOh.jpg)

### Focus: Research question(s) and scope
Per the __context__ defined in the [README document](README.md), the focus of this first test is to understand (with minimal investment of time) the given technology, essentially:

1. the Flower Power Soil sensor;
2. the Flower Power app for iPhone;
3. the data files that flow from iPhone app to cloud, and back to user desktop via Python script developed & maintained by GROW Observatory; and
4. the GROW observatory hub website, as a source of end-user information and support.

![img source](https://i.imgur.com/RdGdMxEh.jpg)
![img source](https://i.imgur.com/BPCkrhuh.jpg)

Given that: 

- The Flower Power sensor and app (see images above, as deployed) work together essentially as advertised, though there are some doubts about hardware and software robustness and support [1]; 
- The [GROW Observatory website](https://hub.growobservatory.org/my-data/sensors/) (NB: login required) gives a somewhat useful but limited view of sensor data (see image just below[2]) and [discussion forums](https://hub.growobservatory.org/discussions) that have yet to shape-up into a very useful source of collegial exchange/ support.

![img source](https://i.imgur.com/l2XOgOGh.png)

Moreover: Full deployment of 320 sensors in the manner prescribed by GROW (in [end-user documentation](GROW-Training-Manual.pdf)), including maintenance of devices in the field (assuming some non-zero failure rate, and battery change every 6 months) along with periodic (biweekly is prescribed) data acquisition and uploading for the (3 years) duration of project will require a significant investment on part of this GROW site.

So: Our focus at this point is on the data files -now holding raw data uploaded from all our sensors, downloadable on demand as a set of .csv files (one per sensor) - and figuring out how the information held in these files can be leveraged to add value to our farm operations.

**Preliminary Questions**: A few assumptions that must be validated before we can do anything useful with this data, i.e.:
- DataFlow: Can we in fact download the data easily enough, in a form that we can work with using Pandas? (a Python library: as Python is the language of data tranfer script, we choose that over R, although that is also a good choice for data munging/ analysis);
- Data Model: What indicators are available for tracking? Using what units of measure? Can records be correlated, as time-series data, with data from other sources (e.g. local weather)?
- Data Latency:  From time of upload, how quickly is data available for download?  And what if any is the maxium latency between time-stamp of last record, and time of data upload?
- Data Integrity: Is the data complete (since sensor boot-up time, with no null values) and accurate (does it appear to make sense, as compared with what's known about local conditions)?

SCOPE: For the purposes of this preliminary analysis, data from one sensor (implanted 1 month ago) will be sufficient.  Data from several additional sensors that have since been implanted will be included in subsequent analysis (test2), once the questions above have been answered.

### Data Import & Hygiene
Having downloaded the data (easily enough, using the python script supplied by GROW observatory tech support), the first step is to import our data -along with any tools needed to process it- and run a few basic checks:

In [ ]:
import pandas as pd   #load the analytics software library
df=pd.read_csv("Flower power 6A22.csv")   # Import the .csv datafile from the "test1" sensor
print("Pandas version ",pd.__version__)  # confirm software version, to help ensure reproducibility of results
print("Number of Rows, Columns = ",df.shape)  #to know the depth & scope of dataframe
print("Column Headers: ",df.columns)  # to see names of column headers
print("Datatype of index = ",type(df.index[0]))  # to confirm datatype of the index

**At first glance:** Considering 2866 records (which pandas ingested without a burp) from a single sensor over 4 weeks... This indicates a pretty hi-frequency sampling interval.
Moreover: If datetime is taken as the 1st column, with an index column of type integer outside that frame, this means that pandas is using its own assigned sequential as the index... But that does not support our purposes so well.
What we want is to treat this as time-series data, so that we can corelate data from different sources (i.e. sensors [2] -but also local weather data, and perhaps weather & sensor data from other GROW sites) along the axis of time.
So: we will create a script to take the datetime field as our index, first converting it to the "Timestamp" datatype, which lets us leverage some powewrful methods that pandas ascribes to this datatype.

In [ ]:
# re-Import the .csv datafile from the "test1" sensor, 
# convert the "capture_datetime_utc" column to parse those strings into useful datetime format
# change index to be that column (versus the assigned ID that is pandas default)

df=pd.read_csv("Flower power 6A22.csv",parse_dates=["capture_datetime_utc"], index_col="capture_datetime_utc")

# now ask pandas again to confirm details specified above, i.e.:
print("Number of Rows, Columns = ",df.shape)  #to know the depth & scope of dataframe
print("Column Headers: ",df.columns)  # to see names of column headers
print("Datatype of index = ",type(df.index[0]))  # to confirm datatype of the index
print("Check dataframe for null values: ",df.isnull().values.any())

**CHECK:** As proved by these 4 reports, Pandas has fixed the dataframe as requested, and it is complete (i.e. devoid of null values).  So we can proceed to explore the data in more depth.

### Exploring the Data

As usual for a dataframe of any size too large for human comprehension, we'll do three things:
- inspect the first few records
- inspect the last few records
- run the "describe" method, which provides a few key Descriptive Statistics

In [ ]:
df.head()   #or df.iloc[0:5] , gives first 5 records in either case

In [ ]:
df.tail()  #last 5 records, to confirm timestamp of last update and continuity of update intervals

In [ ]:
df.describe()  #gives a summary of descriptive statistics for the dataframe across all columns

**Evaluation:** Let us consider what this tells us about the data:
- **Data model:** These 4 metrics are indeed what we are interested to monitor. As to the units: degrees celsius for air temperature is perfectly comprehensible, and soil moisture as a percentage makes sense... But the light column does not specify units, though it is labelled as both "live lux" and "average klux" on the Flower Power app.  As for fertilizer level: this one is a mystery, as no hint is given anywhere in the software as to units of measure or how values are calculated.
- **Time dimension:**  The data appears to have been logged at precise 15 minute intervals, using UTC (Co-Ordinated Universal Time) values -that is one hour behind our local time in Portugal (easily converted by pandas on-the-fly, if we specify our locality).  As to latency: there appears to be no appreciable delay between the time when data is uploaded and when it becomes available for download, but then the timestamp of last record is a few hours old.  Also: some hours will elapse before it shows up on the GROW observatory website.
- **Data integrity/ accuracy:** In the absence of any reference values[3], about all we can do is a common-sense reasonability check.  Considering: 
    - _*Temperature:*_ Average (i.e. mean) temperature seems reasonable for the last month, as does the minimum value.  The maximum value is surprising;  hard to imagine how temperature around this sensor could have reached almost 55 degrees, even on the hottest day.  But considering the values at 25/50/75 percentile rank, and the standard deviation, it seems the values follow a reasonably normal distribution, so that max value looks like an anomaly we should perhaps examine more closely.
    - _*Soil Moisture:*_ The percentages seems to fall within a reasonable range, considering that there were a couple of hydration events; this being probably the most critical indicator for our purposes, we'll dig into this one more deeply in a moment.
    - _*Light:*_ I don't know enough about light measurement to say if these values are reasonable, but what i can say is that the meter value drops in real-time as my shadow falls over the sensor, and the graph displays peaks and valleys every day and night (respectively), as you would expect (also true for the Temperature reading, NB).
    - _*Fertilizer Level:*_ Don't know what to make of this one... But it is not so critical for immediate purposes.

So: Soil Moisture being the metric that we're most concerned to monitor, let's look at those values in a comprehensible form -i.e. daily average, probably good enough.

In [ ]:
#average moisture reading as of the last full day (NB: July 18 was partial, as data was downloaded that afternoon.)
print("last full day's average soil moisture % = ",df["2018-07-17"].soil_moisture_percent.mean())
print ("")
#daily average moisture readings since sensor bootup
print ("daily average soil moisture % for the last month = (date | value) : ")
df.soil_moisture_percent.resample('D').mean()

In [ ]:
%matplotlib inline
df.soil_moisture_percent.resample('D').mean().plot(kind='line')
# a graphical plot of those daily average moisture values

These values seem reasonable (although dangerously low, according to the Flower Power app [4]), and do reflect some quite significant changes, which is reasonable, considering that there were several hydration events -i.e. watering, which i recall doing twice, the last of which was in the pm of July 14- and also some overcast weather (with perhaps a few drops of rain?) in the middle of this period, which would make for a temporary countercurrent in the prevailing trend toward dryness.

For more insight, let's pull a graph of the fully granular (i.e. 15 minute interval) data.

In [ ]:
%matplotlib inline
df.soil_moisture_percent.plot(kind='line')
# a plot of that moisture history with full granularity (i.e. 15-min intervals)

Interesting!  Note the extreme spike in the middle of the graph... Indicating one brief watering event directly on the sensor, but the entire potted plant bed was not irrigated so much as in the other 2 periods of higher humidity.

### CONCLUSIONS: notes and questions for further study

#### Notes:
[1] There is some doubt about impermeability of the battery compartment, per [Amazon product reviews](https://www.amazon.com/gp/customer-reviews/R3LHX6X69XHFR/ref=cm_cr_dp_d_rvw_ttl?ie=UTF8&ASIN=B00FOM2Y6W), along with other issues (esp. problems after firmware update) [discussed in the online user support forums](https://community.parrot.com/t5/Flower-Power/Flower-Power-fails-to-read-mositure-after-update-to-v2-02/m-p/149164) hosted by Parrot company -OEM of the Flower Power hardware and software, who has [officially abandoned support of the product](https://community.parrot.com/t5/Flower-Power/End-of-support-Removing-Parrot-Flower-Power-from-IFTTT/m-p/165883).  Though GROW Observatory appears to have assumed responsibility for support, it is unclear at this point if this extends to the mobile app and/or the sensor hardware and firmware.

[2] Regarding the [sensor visualization page](https://hub.growobservatory.org/my-data/sensors/) and its current limitations, there is now [a forum thread describing the issues](https://hub.growobservatory.org/discussion/sensor-data-visualization-page), along with response from GROW Observatory support team indicating that they are aware of the issues (still open as of this writing) and are working to solve them.

[3] We are getting data from each sensor as its own .csv file... Which means that data from these files must be joined, in order to analyse conditions across sensors (and different GROW sites, potentially) along the axis of time.  CAUTION:  this could be an expensive join!  We may want to concatenate these files, for the purposes of such analysis. 

[4] The Flower Power app contains a database of some >6k plants, whose needs are taken into account in giving to the user information about whether plant is getting too little or too much water/ fertilizer/ light/ temperature (could be very handy for JIT -i.e. precision agriculture- interventions).  This feature does not appear to be reflected in the "My Sensors" page at GROWobservatory.org.

#### Remaining Questions:

1. Sensor Fidelity: to confirm accuracy of readings, can we replicate results across two sensors placed in essentially identical conditions, with sensor boot-up at essentially the same time?
2. Timing: to collect and upload data from a single sensor, proceding with minimal delay to the next, repeat ad (n)finitum: how much time per sensor-visit is required? to what extent can we minimze latency between these operations? and
3. How can we correlate data along the time dimension? rounding off to the nearest common interval?

#### Next Steps:
Considering the importance of these questions, as they relate to the farm's primary motivation in this project (i.e. using the gathered data to inform better farm management decisions), it seems prudent to get them answered -again, with minimal investment of resources- before scaling up to full deployment.  So we will proceed to conduct a second test, using several more sensors placed for ease of deployment/ monitoring on the farm -documentation of same to be posted at [the github repository](https://github.com/ludwa6/soil_sensing) that host's this project's documentation.